# Actualización de Drupal 8

Muchos de las aplicaciones Web incluyendo los CMSs tienen la capacidad de actualizarse vía Web con apenas un par de clicks, lo que hace muy sencilla la adminstración de unos cuantos sitios web.

Es importante evitar aplicaciones donde la actualización es un proceso manual delicado, o por lo menos valerse de prácticas de actualización y de herramientas automatizadas como Ansible o Puppet para asistirse en el proceso.

En el caso de Drupal, la actualización no se realiza vía Web sino mediante la herramienta de ``composer(1)`` la cual permite realizar operaciones sobre cientos a miles de sitios de Drupal utilizando la línea de comando o herramientas de automatización lo que permite escalar la administración de sitios a nivel de proveedor de servicios.

### Instalación de Composer

Iniciamos por instalar la herramienta de Drush en nuestro servidor con Drupal 8.

In [ ]:
cd ~
sudo apt-get -y install curl php-cli php-mbstring git unzip
curl -sS https://getcomposer.org/installer -o composer-setup.php
php -r "if (hash_file('SHA384', 'composer-setup.php') === '669656bab3166a7aff8a7506b8cb2d1c292f042046c5a994c43155c0be6190fa0355160742ab2e1c88d40d5be660b410') { echo 'Installer verified'; } else { echo 'Installer corrupt'; unlink('composer-setup.php'); } echo PHP_EOL;"
sudo php composer-setup.php --install-dir=/usr/local/bin --filename=composer


### Iniciando la actualización

Parte del proceso que vamos a obviar por razones de tiempo y simplicidad, son todas las tareas que se deben realizar antes de realizar un proceso de actualización sobre un servidor en producción como lo son:

- [ ] Actualización de sistema operativo
- [ ] Respaldo de base de datos
- [ ] Respaldo de archivos de distribución
- [ ] Respaldo de personalizaciones de configuración de servidor Web o infraestructura LAMP
- [ ] Práctica de actualización en servidores de pruebas
- [ ] Revisión de compatibilidad de módulos y temas.

Así mismo el proceso de actualización de versiones **mayores** puede que incluya pasos adicionales, así como para versiones mayores de distribución de Sistema Operativo.

Comenzamos por poner el servidor de modo de mantenimiento, lo cual es importante porque es **visible** a nivel de los usuarios.


In [ ]:
sudo apt-get install acl
sudo setfacl -Rm greencore:rwX /var/www/
cd /var/www/html
composer outdated

### Iniciando la actualización

**Importante:** En producción se recomienda actualizar los módulos de forma manual, utilizando **``composer update drupal/modulename --with-dependencies``**, sin embargo en esta guía vamos a actualizar la totalidad de Drupal.

In [ ]:
cd /var/www/html
composer update --with-dependencies

Descomprimimos el código de Drupal dentro de /var/www/html que es el directorio default de Apache.

En un sistema en producción se recomienda crear un directorio para cada virtualhost, o un único virtualhost para drupal y hacer uso de la función de "Multi tenant" para trabajar varios clientes con una sola instancia de la aplicación.

Importante que el código queda con derechos de **www-data**, usuario que utiliza Debian para el servicio de web de Apache. Esto porque Drupal debe poder manipular archivos dentro de la carpeta.

In [ ]:
sudo composer global require drush/drush:8.x
sudo /home/greencore/.composer/vendor/bin/drush updb
sudo /home/greencore/.composer/vendor/bin/drush cr